In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
import scanpy.api as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()

/home/ahsvargo/miniconda3/envs/r35py37/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


scanpy==1.3.7+56.gde16b79.dirty anndata==0.6.11 numpy==1.14.6 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.0 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


In [4]:
import sys
sys.path.append('/home/ahsvargo/xvalid')
sys.path.append('/home/ahsvargo/.local/bin')

In [5]:
from picturedrocks import Rocks
from picturedrocks.performance import FoldTester, PerformanceReport, NearestCentroidClassifier

In [6]:
import random

In [7]:
import loadData as ld

scanpy==1.3.7+56.gde16b79.dirty anndata==0.6.11 numpy==1.14.6 scipy==1.1.0 pandas==0.23.4 scikit-learn==0.20.0 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


## Random Forest Classifier

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [9]:
class RandomForest:
    def __init__(self):
        self.traindata = None
        self.RFC = RandomForestClassifier(n_estimators=100)
        
    def train(self, data):
        self.traindata = data
        self.RFC.fit(data.X, data.y[:,0])
        
    def test(self, Xtest, sparse):
        return self.RFC.predict(Xtest)
    

## Load datasets
---
### Paul

In [8]:
adata = sc.datasets.paul15()

... storing 'paul15_clusters' as categorical


In [9]:
lookup = list(adata.obs['paul15_clusters'].cat.categories)
y = np.array([lookup.index( adata.obs['paul15_clusters'][i] ) for i in range(adata.obs['paul15_clusters'].shape[0]) ])

In [10]:
data = Rocks(adata.X, y)
ft = FoldTester(data)
folds = np.load("paul15-scviFolds.npz")["folds"]
ft.folds = folds
ft.validatefolds()

ft.makerocks(verbose=0)

0

### Zeisel

In [21]:
rawdata = np.load("zeisel/zeisel-proc.npz")

In [22]:
data = Rocks(rawdata['X'], rawdata['y'])
ft = FoldTester(data)
ft.loadfolds('zeisel/zeisel14-5folds.npz')

ft.makerocks(verbose=0)

0

### Zheng

In [10]:
path= "/home/ahsvargo/turbo/scData/zheng17/filtered_matrices_mex/hg19/"

In [11]:
allGenes=True
labels='bulk'
data, ft = ld.load_zheng('Rocks', labels=labels, allGenes=allGenes)

## Choose random genes to be markers

Maybe can restrict to those that are highly expressed or highly variable if this is producing too much noise

In [12]:
def randomMarkers(rocks):
    
    markers = []
    dim = rocks.X.shape[1] # number of genes
    
    for clust in np.unique(rocks.y):
        markers.append(np.random.permutation(dim)) #lol
        
    return np.array(markers)

In [15]:
%%time
#lol

randomMarkerList = []
for rock in ft.rocks:
    randomMarkerList.append(randomMarkers(rock))
    
randomMarkerList = np.array(randomMarkerList)

np.savez("zheng-bulk-randomMarkers.npz", randomMarkerList)
    

CPU times: user 37.9 ms, sys: 24.9 ms, total: 62.7 ms
Wall time: 170 ms


In [16]:
randomMarkerList.shape

(5, 11, 20387)